Fonte: https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/

In [1]:
!pip install transformers beautifulsoup4 unidecode

     |████████████████████████████████| 2.0MB 5.5MB/s 
     |████████████████████████████████| 245kB 17.8MB/s 
     |████████████████████████████████| 890kB 12.5MB/s 
     |████████████████████████████████| 3.2MB 26.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3990150a943fada9c7537b8d803425fb83a0e5060fcdb8cc861081abfe64585d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel, AdamW, get_linear_schedule_with_warmup  # or BertModel, for BERT without pretraining heads

import requests
from unidecode import unidecode
from torch import nn, optim
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import confusion_matrix, classification_report
import torch.nn.functional as F
from collections import defaultdict
from bs4 import BeautifulSoup

RANDOM_SEED = 2021
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
bert_model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

## Tests

In [ ]:
# input_ids = tokenizer.encode('Banco', return_tensors='pt') # pt == pytorch tensors
input_ids = tokenizer.encode('brasil', return_tensors='np', add_special_tokens=False) 
print(input_ids)
# with torch.no_grad():
#     outs = model(input_ids)
#     encoded = outs[0][0, 1:-1]

# print(len(encoded))
# print(encoded)

[[1010  215]]


In [ ]:
sample = "Eu tranferi um dinheiro do Bradesco pra conta da Nubank na forma TED era pra cair na hora e não caiu alguém sabe me explicar do porque ainda não caiu???"
sample = unidecode(''.join(c for c in sample.lower() if c.isalnum() or c == ' '))
print(sample)

eu tranferi um dinheiro do bradesco pra conta da nubank na forma ted era pra cair na hora e nao caiu alguem sabe me explicar do porque ainda nao caiu


In [ ]:
# print(tokenizer.tokenize(sample))
# print(len(tokenizer.tokenize(sample)))

In [ ]:
encoding = tokenizer.encode_plus(
  sample,
  max_length=100,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)
# encoding.keys()

print(len(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

100
tensor([  101,  2779,  8196,   725, 22283,   222,  3495,   171,  4332, 19813,
         1174,  1284,   180,  1444,  5319, 22331,   229,   547,   437, 22284,
          495,  1174,  9322,   229,  5314,   122,   229, 22280,  6600,   614,
          210,  4178,   311,  9718,   171,  2113,   744,   229, 22280,  6600,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
with torch.no_grad():
    last_hidden_states = bert_model(encoding['input_ids'], attention_mask=encoding['attention_mask'])

last_hidden_states

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[ 0.1523, -0.4904,  0.2555,  ..., -0.5522,  0.1028, -0.3075],
                                                        [ 0.3485, -0.3699,  0.3692,  ...,  0.1952,  0.0594, -0.1596],
                                                        [ 0.1378, -0.2091,  0.9594,  ..., -0.1914, -0.2921, -0.8497],
                                                        ...,
                                                        [ 0.2756, -0.4099,  0.3662,  ...,  0.4792, -0.2992, -0.4784],
                                                        [ 0.1909, -0.2545,  0.3394,  ...,  0.4693, -0.2626, -0.3726],
                                                        [ 0.4592, -0.3135,  0.4242,  ...,  0.3944, -0.0203, -0.5740]]])),
                                              ('pooler_output',
                                               tensor([[-1.8347e-01,  7.5434e-02, -8.0949e-02, 

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
print(features)

[[ 1.52250811e-01 -4.90394473e-01  2.55453438e-01 -4.55293059e-01
   6.84672415e-01  4.40870136e-01 -1.14103325e-01  4.14126217e-01
   1.04695126e-01  5.05802572e-01 -2.38476366e-01  3.74412000e-01
  -1.14908725e-01 -8.83286834e-01  9.12145525e-02  8.89936313e-02
  -3.42116326e-01  3.58302325e-01  1.86703466e-02  4.47669625e-02
  -4.03186977e-01  2.94000715e-01  9.96866971e-02  1.15009539e-01
   1.73238441e-01 -3.14020403e-02 -4.77160633e-01  5.98942578e-01
   5.85707724e-01  7.35906214e-02 -1.23555586e-01  4.16242778e-01
   3.13680947e-01  1.18071258e-01 -7.12796152e-01 -5.06621659e-01
  -1.77628309e-01 -4.54526320e-02  1.28552049e-01 -1.77131549e-01
   3.87519270e-01  4.26965982e-01  1.23904392e-01  1.26919642e-01
  -4.02285792e-02 -1.22478768e-01 -3.26906353e-01 -9.57922116e-02
  -1.83053643e-01  3.65122169e-01  1.24855270e-03 -1.90708011e-01
  -4.75983679e-01  1.37353852e-01  2.17017576e-01 -3.52647781e-01
   1.70543388e-01 -4.22987416e-02 -1.78507030e-01  2.27408290e-01
  -6.38377

In [ ]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

## Gerar dataset

In [4]:
def get_max_len(dataset):
    return len(max(dataset["text"], key=lambda s: len(s)))

def encode(text, tokenizer, max_len, return_tensors='np'):
    return tokenizer.encode_plus(
        text,
        max_length=min(512, max_len),
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors=return_tensors
    )

Dataset para o sklearn

In [5]:
def PageDatasetSklearn(dataset, tokenizer, model):
    '''Use example:
    X, y = PageDatasetSklearn(nubank, tokenizer, bert_model)'''

    df = dataset.copy()

    max_len = get_max_len(df)

    texts = list(df['text'])

    texts = map(lambda t: encode(t, tokenizer, max_len), texts)
    
    texts = list(texts)
    
    # A primeira declaracao retorna uma lista de lista de tamanho 1 com a frase codificada: [[['oi']], [['opa']]]
    # A segunda retira essa lista de cada lista: [['oi'], ['opa']]
    input_ids = [t['input_ids'] for t in texts]
    input_ids = [list(x[0]) for x in input_ids]
    attention_mask = [t['attention_mask'] for t in texts]
    attention_mask = [list(x[0]) for x in attention_mask]

    with torch.no_grad():
        last_hidden_states = model(torch.tensor(input_ids), attention_mask=torch.tensor(attention_mask))
    
    # Pega só a saida do primeiro token [CLS] que vai servir para a classificacao
    features = last_hidden_states[0][:,0,:].numpy()
    return features, df['target']



Dataset e data loader para o pytorch

In [6]:
class PageDatasetTorch(Dataset):

    def __init__(self, dataset, tokenizer):
        self.texts = dataset['text'].to_numpy()
        self.targets = dataset['target'].to_numpy()
        self.tokenizer = tokenizer
        self.max_len = min(get_max_len(dataset), 512)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        target = self.targets[item]

        encoding = encode(text, self.tokenizer, self.max_len)

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [7]:
def create_data_loader(dataset, tokenizer):
    BATCH_SIZE = 1
    df = PageDatasetTorch(dataset, tokenizer)

    return DataLoader(
        df, batch_size=BATCH_SIZE, num_workers=4
    )

Chamar `create_data_loader` para o conjunto de treino, validacao e teste

## Modelo

In [ ]:
# from google.colab import files
# uploaded = files.upload()
# import io
# dataset = pd.read_csv(io.BytesIO(uploaded['nubank.csv']))

Saving nubank.csv to nubank.csv


In [8]:
nubank = pd.read_csv('./nubank.csv')
nuvem = pd.read_csv('./nuvem.csv')
shopify = pd.read_csv('./shopify.csv')
vtex = pd.read_csv('./vtex.csv')

### Pytorch

In [ ]:
# data = next(iter(train_data_loader))
# data.keys()

In [9]:
class PageClassifier(nn.Module):

    def __init__(self, n_classes):
        super(PageClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(last_hidden_states[1])
        return self.out(output)

In [ ]:
df_train, df_test = train_test_split(dataset, test_size=0.33, random_state=RANDOM_SEED)

In [ ]:
train_data_loader = create_data_loader(df_train, tokenizer)
test_data_loader = create_data_loader(df_test, tokenizer)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#### Como rodar

In [ ]:
data = next(iter(train_data_loader))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

In [11]:
PyTorchModel = PageClassifier(2)
PyTorchModel = PyTorchModel.to(device)

In [ ]:
F.softmax(PyTorchModel(input_ids, attention_mask), dim=1)

tensor([[0.4009, 0.5991]], device='cuda:0', grad_fn=<SoftmaxBackward>)

#### Treinando

In [13]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [14]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
EPOCHS = 4

optimizer = AdamW(PyTorchModel.parameters(), lr=3e-4, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    PyTorchModel,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

#   val_acc, val_loss = eval_model(
#     PyTorchModel,
#     val_data_loader,
#     loss_fn, 
#     device, 
#     len(df_val)
#   )

#   print(f'Val   loss {val_loss} accuracy {val_acc}')
#   print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
#   history['val_acc'].append(val_acc)
#   history['val_loss'].append(val_loss)

  if train_acc > best_accuracy:
    torch.save(PyTorchModel.state_dict(), 'best_model_state.bin')
    best_accuracy = train_acc

Epoch 1/4
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train loss 3.368763352540327e-05 accuracy 1.0
Epoch 2/4
----------
Train loss 0.0 accuracy 1.0
Epoch 3/4
----------
Train loss 0.0 accuracy 1.0
Epoch 4/4
----------
Train loss 0.0 accuracy 1.0
CPU times: user 4.97 s, sys: 3.62 s, total: 8.58 s
Wall time: 9.39 s


#### Avaliacao

In [ ]:
test_acc, _ = eval_model(
  PyTorchModel,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)
test_acc.item()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1.0

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_texts, y_pred, y_pred_probs, y_test = get_predictions(
  PyTorchModel,
  test_data_loader
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         2

    accuracy                           1.00         7
   macro avg       1.00      1.00      1.00         7
weighted avg       1.00      1.00      1.00         7



#### Cross validation

In [21]:
def cross_val_pytorch(dataset, num_epochs=4):


    kfold = KFold(n_splits=4, shuffle=True)

    # df_train, df_test = train_test_split(dataset, test_size=0.25, random_state=RANDOM_SEED)

    # train_scores = []
    test_scores = []

    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        train = dataset.iloc[train_ids]
        test = dataset.iloc[test_ids]

        train_data_loader = create_data_loader(train, tokenizer)
        test_data_loader = create_data_loader(test, tokenizer)

        network = PageClassifier(2)
        network = network.to(device)

        total_steps = len(train_data_loader) * num_epochs

        optimizer = AdamW(network.parameters(), lr=3e-4, correct_bias=False)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )

        loss_fn = nn.CrossEntropyLoss().to(device)

        # epoch_scores = []

        for epoch in range(num_epochs):
            train_acc, train_loss = train_epoch(
                network,
                train_data_loader,    
                loss_fn, 
                optimizer, 
                device, 
                scheduler, 
                len(nubank_train)
            )

            # epoch_scores.append(train_acc)

        test_acc, _ = eval_model(
            network,
            test_data_loader,
            loss_fn,
            device,
            len(nubank_test)
        )

        test_scores.append(test_acc)

        torch.cuda.empty_cache()
    
    return test_scores

In [19]:
num_epochs=4

kfold = KFold(n_splits=4, shuffle=True)

df_train, df_test = train_test_split(nubank, test_size=0.25, random_state=RANDOM_SEED)

# train_scores = []
test_scores = []

for fold, (train_ids, test_ids) in enumerate(kfold.split(nubank)):
    # print(test_ids)
    nubank_train = nubank.iloc[train_ids]
    nubank_test = nubank.iloc[test_ids]
    # print(nubank_train)

    train_data_loader = create_data_loader(nubank_train, tokenizer)
    test_data_loader = create_data_loader(nubank_test, tokenizer)

    network = PageClassifier(2)
    network = network.to(device)

    total_steps = len(train_data_loader) * num_epochs

    optimizer = AdamW(network.parameters(), lr=3e-4, correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    loss_fn = nn.CrossEntropyLoss().to(device)

    epoch_scores = []

    for epoch in range(num_epochs):
        train_acc, train_loss = train_epoch(
            network,
            train_data_loader,    
            loss_fn, 
            optimizer, 
            device, 
            scheduler, 
            len(nubank_train)
        )

        epoch_scores.append(train_acc)

    test_acc, _ = eval_model(
        network,
        test_data_loader,
        loss_fn,
        device,
        len(nubank_test)
    )

    torch.cuda.empty_cache()

    test_scores.append(test_acc)



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [20]:
print(test_scores)

[tensor(0.4000, device='cuda:0', dtype=torch.float64), tensor(0.4000, device='cuda:0', dtype=torch.float64), tensor(0.6000, device='cuda:0', dtype=torch.float64), tensor(0.2000, device='cuda:0', dtype=torch.float64)]


In [25]:
nubank_cross_val_scores = cross_val_pytorch(nubank)
print([x.item() for x in nubank_cross_val_scores])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0.2, 0.4, 0.6000000000000001, 0.4]


In [22]:
nuvem_cross_val_scores = cross_val_pytorch(nuvem)
print(nuvem_cross_val_scores)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[tensor(0.6000, device='cuda:0', dtype=torch.float64), tensor(0.2000, device='cuda:0', dtype=torch.float64), tensor(0.8000, device='cuda:0', dtype=torch.float64), tensor(0.6000, device='cuda:0', dtype=torch.float64)]


In [24]:
print([x.item() for x in nuvem_cross_val_scores])

[0.6000000000000001, 0.2, 0.8, 0.6000000000000001]


In [26]:
shopify_cross_val_scores = cross_val_pytorch(shopify)
print([x.item() for x in shopify_cross_val_scores])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0.0, 0.2, 0.8, 0.4]


In [27]:
vtex_cross_val_scores = cross_val_pytorch(vtex)
print([x.item() for x in vtex_cross_val_scores])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0.2, 0.4, 0.8, 0.4]


### Sklearn

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#### Nubank

In [ ]:
X_nubank, y_nubank = PageDatasetSklearn(nubank, tokenizer, bert_model)



In [ ]:
X_train_nubank, X_test_nubank, y_train_nubank, y_test_nubank = train_test_split(X_nubank, y_nubank, 
                                test_size=0.33, random_state=RANDOM_SEED)

In [ ]:
svc = SVC()

print(cross_val_score(svc, X_nubank, y_nubank, cv=4, scoring='f1'))

[0.8        1.         0.85714286 1.        ]


In [ ]:
rfc = RandomForestClassifier(max_depth=10)

print(cross_val_score(rfc, X, y, cv=4, scoring='f1'))

[0.8        0.8        0.8        0.85714286]


#### nuvem

In [ ]:
X_nuvem, y_nuvem = PageDatasetSklearn(nuvem, tokenizer, bert_model)

In [ ]:
X_train_nuvem, X_test_nuvem, y_train_nuvem, y_test_nuvem = train_test_split(X_nuvem, y_nuvem, 
                                test_size=0.33, random_state=RANDOM_SEED)

In [ ]:
svc = SVC()

print(cross_val_score(svc, X_nuvem, y_nuvem, cv=4, scoring='f1'))

[0.66666667 0.8        0.8        0.85714286]


In [ ]:
rfc = RandomForestClassifier(max_depth=10)

print(cross_val_score(rfc, X_nuvem, y_nuvem, cv=4, scoring='f1'))

[1.  1.  0.8 1. ]


#### shopify

In [ ]:
X_shopify, y_shopify = PageDatasetSklearn(shopify, tokenizer, bert_model)

In [ ]:
X_train_shopify, X_test_shopify, y_train_shopify, y_test_shopify = train_test_split(X_shopify, y_shopify, 
                                test_size=0.33, random_state=RANDOM_SEED)

In [ ]:
svc = SVC()

print(cross_val_score(svc, X_shopify, y_shopify, cv=4, scoring='f1'))

[0.8        0.8        0.85714286 1.        ]


In [ ]:
rfc = RandomForestClassifier(max_depth=10)

print(cross_val_score(rfc, X_shopify, y_shopify, cv=4, scoring='f1'))

[1.         0.8        0.85714286 1.        ]


#### VTEX

In [ ]:
X_vtex, y_vtex = PageDatasetSklearn(vtex, tokenizer, bert_model)

In [ ]:
X_train_vtex, X_test_vtex, y_train_vtex, y_test_vtex = train_test_split(X_vtex, y_vtex, 
                                test_size=0.33, random_state=RANDOM_SEED)

In [ ]:
svc = SVC()

print(cross_val_score(svc, X_vtex, y_vtex, cv=4, scoring='f1'))

[0.8        0.66666667 0.         0.85714286]


In [ ]:
rfc = RandomForestClassifier(max_depth=10)

print(cross_val_score(rfc, X_vtex, y_vtex, cv=4, scoring='f1'))

[0.66666667 0.8        1.         0.85714286]


### Harvest Ratio

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
from utils import get_parsed_phrase, get_parsed_vtex
from selenium import webdriver
import time

In [ ]:
def classify_site(soup, bert_model, model, tokenizer):

    parsed_phrase = get_parsed_phrase(soup)
    if len(parsed_phrase) > 250:
        parsed_phrase = parsed_phrase[:250]
    encoding = encode(parsed_phrase, tokenizer, len(parsed_phrase), return_tensors='pt')
    # print(encoding['input_ids'][0])
    with torch.no_grad():
        last_hidden_states = bert_model(encoding['input_ids'], attention_mask=encoding['attention_mask'])

    features = last_hidden_states[0][:,0,:].numpy()
    return model.predict(features)

In [ ]:
def classify_vtex(title, texts, bert_model, model, tokenizer):
    try:
        print(title)
        parsed_phrase = get_parsed_vtex(title, texts)
        if len(parsed_phrase) > 250:
            parsed_phrase = parsed_phrase[:250]
        encoding = encode(parsed_phrase, tokenizer, len(parsed_phrase), return_tensors='pt')
        with torch.no_grad():
            last_hidden_states = bert_model(encoding['input_ids'], attention_mask=encoding['attention_mask'])

        features = last_hidden_states[0][:,0,:].numpy()
        return model.predict(features)
    except ValueError:
        print(title)
        return [0]

In [ ]:
def get_links_list(filename):
    raw = open(f'{filename}.txt', 'r')

    links = []
    actual = raw.readline()

    while actual != "":
        actual = actual.rstrip('\n')
        if actual != '':
            links.append(actual)
        actual = raw.readline()

    return links

In [ ]:
def get_predictions(model, links):
    predicts = []

    for link in links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        predicts.append(classify_site(soup, bert_model, model, tokenizer))

    return predicts

In [ ]:
def get_vtex_predictions(model, links):
    predicts = []

    for link in links:
        options = webdriver.ChromeOptions()
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--incognito")
        options.headless = True
        options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=options)
        driver.get(link)
        time.sleep(5)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        blacklist = [
            'style',
            'script'
        ]
        text_elements = [t for t in soup.find_all(
            text=True) if t.parent.name not in blacklist]
        text_elements = list(filter(lambda x: not (
            'render facet' in x or '\n' in x or '\xa0' in x), text_elements))
        
        driver.close()

        predicts.append(classify_vtex(soup.find('title').text, text_elements, bert_model, model, tokenizer))

    return predicts

#### Nubank

In [ ]:
nubank_links = get_links_list('nubank')

In [ ]:
svc_nubank = SVC()

svc_nubank.fit(X_train_nubank, y_train_nubank)

nubank_heuristics = get_predictions(svc_nubank, nubank_links[:100])

print(nubank_heuristics.count([1]))

76


In [ ]:
nubank_bfs = get_predictions(svc_nubank, nubank_links[100:])
print(nubank_bfs.count([1]))

70


#### Nuvem

In [ ]:
nuvem_links = get_links_list('nuvemshop')

In [ ]:
rfc_nuvem = RandomForestClassifier(max_depth=10)

rfc_nuvem.fit(X_train_nuvem, y_train_nuvem)

nuvem_heuristics = get_predictions(rfc_nuvem, nuvem_links[:100])

print(nuvem_heuristics.count([1]))

79


In [ ]:
nuvem_bfs = get_predictions(rfc_nuvem, nuvem_links[100:])
print(nuvem_bfs.count([1]))

49


#### Shopify

In [ ]:
shopify_links = get_links_list('shopify1')

In [ ]:
rfc_shopify = RandomForestClassifier(max_depth=10)

rfc_shopify.fit(X_train_shopify, y_train_shopify)

shopify_heuristics = get_predictions(rfc_shopify, shopify_links[:100])

print(shopify_heuristics.count([1]))

27


In [ ]:
shopify_bfs = get_predictions(rfc_shopify, shopify_links[100:])
print(shopify_bfs.count([1]))

27


#### VTEX

In [ ]:
vtex_links = get_links_list('vtex')

In [ ]:
rfc_vtex = RandomForestClassifier(max_depth=10)

rfc_vtex.fit(X_train_vtex, y_train_vtex)

vtex_heuristics = get_vtex_predictions(rfc_vtex, vtex_links[:100])

print(vtex_heuristics.count([1]))

89


In [ ]:
vtex_bfs = get_vtex_predictions(rfc_vtex, vtex_links[100:])
print(vtex_bfs.count([1]))

VTEX Community
VTEX Developer Portal
VTEX Community
Selo ebit mobile não aparece
VTEX Help Center 
Consigo plugar dois adquirentes de pagamento ao mesmo tempo (no caso Pay U e Mundipagg) e criar uma regra para transacionar entre essas adquirentes por range de produto?
Quero fazer cadastro para ser um ponto de retirada e não consigo
Erro no e-mail
Consigo colocar prazo de entrega diferentes em cada produto?
VTEX Community
VTEX SSO - HELP
Criar coleção automática com todos os produtos da loja
Migração Vtex CMS para Vtex.io
Transação sendo cancelada sozinha
REDIRECT.CENTER está fora do ar ?
Alterar texto vtex.cmc:stockKeepingUnitSelection
Pessoal, sabem se tem algum tempo para o produto fica no Mini cart até sair?
e se quando o produto vai para o carrinho esse produto sai do estoque?
Não consigo fazer o de para de categorias com a Amazon. Meu produto não tem variação de cor e nem tamanho pois são cosmético.
Como compartir un producto en facebook?
Como atualizar as informações de um orderf